this is a markdown comment. the ipynb format is useful for getting individual cells to run

In [3]:
x = "Here we go!"
print(x)

Here we go!


In [12]:
import sys
print(sys.version)


3.13.5 (v3.13.5:6cb20a219a8, Jun 11 2025, 12:23:45) [Clang 16.0.0 (clang-1600.0.26.6)]


In [ ]:
# Cell 1 - Imports and Setup
import requests
import pandas as pd

headers = {'User-Agent': 'aditya.goel.tx@gmail.com'}


In [18]:
# Cell 2 - Load Clean Fund List CSV
df_funds = pd.read_csv("Clean_Fund_List.csv")
df_funds

,fund_group,fund_id,name,ticker,fund_stkid,advt,fund_type
0,fund,04B8BC-E,American Beacon Balanced Fund,ABCCX US,135909,0,MF_O
1,fund,04B8BD-E,American Beacon Large Cap Value Fund,AADEX US,25667,0,MF_O
2,fund,04B8BK-E,RBC Enterprise Fund,TETAX US,84097,0,MF_O
3,fund,04B8BQ-E,Cohen & Steers Real Estate Securities Fund,CREFX US,210588,0,MF_O
4,fund,04B8BY-E,Voya Growth & Income Portfolio,ISVGX US,119362,0,MF_O
...,...,...,...,...,...,...,...
4837,fund,10L5FM-E,Naa World Equity Income Fund,SFGCX US,67247,0,MF_O
4838,fund,10MBT4-E,Kensington Dynamic Growth,KAGCX US,264944,0,MF_O
4839,fund,10ZJSS-E,Vontobel US Equity Fund,VNUYX US,224821,0,MF_O
4840,fund,112M3V-E,New Age Alpha SMid Cap Value Fund,SEVSX US,67504,0,MF_O


In [20]:
# Cell 3 - Pull SEC Company Ticker JSON
url = "https://www.sec.gov/files/company_tickers.json"
company_tickers = requests.get(url, headers=headers).json()

# Convert to DataFrame
df_tickers = pd.DataFrame.from_dict(company_tickers, orient='index')
df_tickers['cik_str'] = df_tickers['cik_str'].astype(str).str.zfill(10)
df_tickers.rename(columns={'title': 'sec_name'}, inplace=True)
df_tickers


,cik_str,ticker,sec_name
0,0001045810,NVDA,NVIDIA CORP
1,0000789019,MSFT,MICROSOFT CORP
2,0000320193,AAPL,Apple Inc.
3,0001018724,AMZN,AMAZON COM INC
4,0001652044,GOOGL,Alphabet Inc.
...,...,...,...
10055,0001949257,RDPTF,Radiopharm Theranostics Ltd
10056,0001733968,WLGMF,West Red Lake Gold Mines Ltd.
10057,0001991946,CGBSW,Crown LNG Holdings Ltd
10058,0002038919,DWWYF,Dowway Holdings Limited/ADR
